# Workshop 12.1 BERT classification




In [ ]:
!pip install datasets transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 25.8 MB/s 
     |████████████████████████████████| 5.8 MB 70.1 MB/s 
     |████████████████████████████████| 212 kB 82.3 MB/s 
     |████████████████████████████████| 132 kB 53.6 MB/s 
     |████████████████████████████████| 182 kB 80.3 MB/s 
     |████████████████████████████████| 127 kB 77.9 MB/s 
     |████████████████████████████████| 7.6 MB 68.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


# BERT

BERT — это нейронная сеть от Google, показавшая с большим отрывом state-of-the-art результаты на целом ряде задач. 

Точно так же, как и в обычном Трансформере, BERT принимает на вход последовательность слов, которая затем продвигается вверх по стеку экодеров. Каждый слой энкодера применяет внутреннее внимание (self-attention) и передает результаты в сеть прямого распространения, после чего направляет его следующему энкодеру.

![](https://habrastorage.org/r/w1560/getpro/habr/post_images/2bd/0ba/1c4/2bd0ba1c4fb80fe4d771f555168c9ff0.png)




https://medium.com/@samia.khalid/bert-explained-a-complete-guide-with-theory-and-tutorial-3ac9ebc8fa7c



Задачи, которые можно решать при помощи BERT

![](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/BERT-tasks.png?ssl=1)

# Hugging Face
https://huggingface.co/

Библиотека + Сообщество по ML


In [ ]:
import transformers

print(transformers.__version__)

4.25.1


In [ ]:
import torch
import torch.nn as nn
from tqdm.notebook import tqdm
import torch.nn.functional as F

# Fine-tuning a model on a text classification task

Давайте возьмем готовую модель BERT и дообучим для предсказания оценки по тексту ревью.

## Loading the dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Посмотрим что лежит внутри

In [ ]:
dataset["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

Рассмотрим несколько случайных примеров

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

## Подготовка данных

Прежде чем мы сможем передать эти тексты нашей модели, нам нужно их предварительно обработать. Это делается с помощью 🤗 Transformers Tokenizer, который токенизирует входные данные (включая преобразование токенов в соответствующие идентификаторы в предварительно обученном словаре) и переводит их в формат, ожидаемый моделью, а также генерирует другие входные данные. эта модель требует.

Что бы модела получала такие же данные что и при обучении, мы будем использовать с такими же параметрами и словарем. 

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)



Применим его

In [ ]:
tokenizer("Hello, this one sentence!")

NameError: ignored

В зависимости от выбранной вами модели `tokenizer` может возвращать разные данные. 

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)


This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 4066, 1997, 4516, 2006, 2054, 1996, 2779, 25430, 14728, 2245, 2055, 3056, 2576, 3314, 2107, 2004, 1996, 5148, 2162, 1998, 2679, 3314, 1999, 1996, 2142, 2163, 1012, 1999, 2090, 4851, 8801, 1998, 6623, 7939, 4697, 3619, 1997, 8947, 2055, 2037, 10740, 2006, 4331, 1010, 2016, 2038, 3348, 2007, 2014, 3689, 38

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
train_data = encoded_dataset['train']

In [ ]:
train_data[0].keys()

dict_keys(['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
pad_index = 0

def collate(batch):
    batch_ids = [torch.tensor(i['input_ids']) for i in batch]
    batch_ids = nn.utils.rnn.pad_sequence(batch_ids, padding_value=pad_index, batch_first=True)
    
    batch_token_type = [torch.tensor(i['token_type_ids']) for i in batch]
    batch_token_type = nn.utils.rnn.pad_sequence(batch_token_type, padding_value=pad_index, batch_first=True)
    
    attention_mask = [torch.tensor(i['attention_mask']) for i in batch]
    attention_mask = nn.utils.rnn.pad_sequence(attention_mask, padding_value=pad_index, batch_first=True)

    batch_label = [i['label'] for i in batch]
    batch_label = torch.tensor(batch_label, dtype=torch.long)

    batch = {'input_ids': batch_ids,
             'token_type_ids': batch_token_type, 
             'attention_mask': attention_mask,
             'labels': batch_label}
    return batch
    

dataloader = torch.utils.data.DataLoader(train_data, 
                                               batch_size=16, 
                                               collate_fn=collate, 
                                               shuffle=True)

In [ ]:
next(iter(dataloader))

{'input_ids': tensor([[  101,  1996,  2069,  ...,     0,     0,     0],
         [  101,  2023,  3185,  ...,     0,     0,     0],
         [  101, 14439, 24665,  ...,     0,     0,     0],
         ...,
         [  101, 11430,  2402,  ...,  4150,  2205,   102],
         [  101,  2045,  2428,  ...,     0,     0,     0],
         [  101,  2610,  2466,  ...,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1])}

## Дообучение модели

Теперь, мы можем загрузить предварительно обученную модель и настроить ее. 

Мы используем класс `AutoModelForSequenceClassification`. Как и в случае с токенизатором, метод `from_pretrained` загружает и кэширует модель для нас. 



In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
model = model.to(device)

batch = next(iter(dataloader))
batch = {k: v.to(device) for k, v in batch.items()}


with torch.no_grad():
    result = model(**batch)
result

SequenceClassifierOutput(loss=tensor(0.7048, device='cuda:0'), logits=tensor([[ 0.2223,  0.1863],
        [ 0.1761,  0.2544],
        [ 0.2534,  0.0823],
        [ 0.3129,  0.0427],
        [ 0.2603,  0.3042],
        [ 0.1445,  0.1831],
        [ 0.2651,  0.1249],
        [ 0.3067,  0.2200],
        [ 0.1624,  0.0541],
        [ 0.1163,  0.1659],
        [ 0.2102, -0.0363],
        [ 0.2370,  0.3051],
        [ 0.2631,  0.0882],
        [ 0.2242,  0.4977],
        [ 0.2954,  0.1043],
        [ 0.2430,  0.3450]], device='cuda:0'), hidden_states=None, attentions=None)

In [ ]:
result = None

In [ ]:
model.train().to(device)

optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)

for i, batch in enumerate(tqdm(dataloader)):
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs[0]
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if i % 20 == 0:
        print(f"loss: {loss}")

    if i == 100:
        break

  0%|          | 0/1563 [00:00<?, ?it/s]

loss: 0.6990270018577576
loss: 0.7110907435417175
loss: 0.6598783731460571
loss: 0.5693095922470093
loss: 0.42444372177124023
loss: 0.5670837163925171


Можно воспользовать готовым решением: `Trainer`

In [ ]:
from datasets import load_metric
import numpy as np
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-23-165db97bb7a9>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


In [ ]:
batch_size = 16
 
args = TrainingArguments(
    "bert-finetuned",
    evaluation_strategy="steps",
    logging_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    logging_steps=20,
    eval_steps=20,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 25000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4689
  Number of trainable parameters = 109483778
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a meth

Step,Training Loss,Validation Loss


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 16


KeyboardInterrupt: ignored

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

In [ ]:
trainer.evaluate()